In [ ]:
#Start of the project
#data preprocessing
#convert data into csv format, also create train set, valid set and test set
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as et
from google.colab import drive
from sklearn.model_selection import train_test_split
drive.mount('/content/gdrive/')

IMAGEPATH = 'gdrive/My Drive/Project1/dataset/images' #path linked to the file storing all images
ANNOPATH = 'gdrive/My Drive/Project1/dataset/annotations' #path linked to the file storing all xml
DATAPATH = 'gdrive/My Drive/Project1/dataset'

features = {'image_name': [], 'width': [], 'height': [], 'xmin': [], 'ymin': [], 'xmax': [], 'ymax': [], 'label': []} #extracted information of each data

for xml in os.listdir(ANNOPATH):
  info = et.parse(ANNOPATH + '/' + xml)
  root = info.getroot()
  image = root[1].text
  width = root[2][0].text
  height = root[2][1].text
  for i in range(4, len(root)):
    label = root[i][0].text
    #with_mask = 0, without_mask = 1, mask_weared_incorrect = 2
    if label == 'with_mask':
      label = 0
    elif label == 'without_mask':
      label = 1
    elif label == 'mask_weared_incorrect':
      label = 2
    cur_data = []
    cur_data.append(image)
    cur_data.append(width)
    cur_data.append(height)
    for coord in root[i][5]:
      cur_data.append(coord.text)
    cur_data.append(label)
    for i, feature in enumerate(features):
      features[feature].append(cur_data[i])

dataset = pd.DataFrame(features)
target_count = dataset['label'].value_counts()
print('Class 0: ', target_count[0])
print('Class 1: ', target_count[1])
print('Class 2: ', target_count[2])
dataset.to_csv(path_or_buf = DATAPATH + '/preprocessed.csv', index = False)

#start splitting into training set, validation set and test set
features = dataset[['image_name', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']]
labels = dataset['label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2, random_state = 4471)
train_features, valid_features, train_labels, valid_labels = train_test_split(train_features, train_labels, test_size = 815, random_state = 4471)

train_features.insert(7, 'label', train_labels)
test_features.insert(7, 'label', test_labels)
valid_features.insert(7, 'label', valid_labels)

train = train_features
test = test_features
valid = valid_features

print(train)
print()
print(test)
print()
print(valid)
print()

train.to_csv(path_or_buf = DATAPATH + '/train.csv', index = False)
test.to_csv(path_or_buf = DATAPATH + '/test.csv', index = False)
valid.to_csv(path_or_buf = DATAPATH + '/valid.csv', index = False)

In [ ]:
#create dataset class
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader


class ImageDataset(Dataset):
    def __init__(self, csv_file, transform = None):
        self.df = pd.read_csv(csv_file)      
        self.transform = transform

    def __getitem__(self, idx):
        image = self.df['image_name'][idx]
        label = self.df['label'][idx]
        
        image = Image.open(IMAGEPATH + '/' + image)
        image = image.convert('RGB') #3 channels

        (left, upper, right, lower) = (int(self.df['xmin'][idx]), int(self.df['ymin'][idx]), int(self.df['xmax'][idx]), int(self.df['ymax'][idx])) #coordinate of bounding box
        image = image.crop((left, upper, right, lower)) #the cropped image, which is the face

        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

    def __len__(self):
        return len(self.df)

In [ ]:
#create dataset and dataloader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize([32, 32])]) # resize to 32 x 32

trainset = ImageDataset(DATAPATH + '/train.csv', transform)
validset = ImageDataset(DATAPATH + '/valid.csv', transform)
testset = ImageDataset(DATAPATH + '/test.csv', transform)

bs = 64

train_loader = DataLoader(trainset, batch_size = bs, shuffle = True)
valid_loader = DataLoader(validset, batch_size = bs, shuffle = False)
test_loader = DataLoader(testset, batch_size = bs, shuffle = False)